In [16]:
import numpy as np
import pandas as pd
%matplotlib inline 
import pandas as pd
import numpy as np

# Load SMILES and features
df = pd.read_csv('data.csv')
print(df)
Xinfo = df.SMILES.values
X = pd.read_csv('features.csv')
print(X)

                                              SMILES       Eox
0                       N#CC(c1ccccc1)C(Br)Oc1ccccc1  2.204905
1    O=[N+]([O-])c1ccc(C(c2ccccc2)C(Br)Oc2ccccc2)cc1  2.171083
2                   CC(Oc1ccccc1)C(C#N)c1ccc(C#N)cc1  1.904724
3                 COC(C#N)C(C)(C)c1ccc(-c2ccccc2)cc1  1.788157
4      COC(Oc1ccccc1)C(C)(C#N)c1ccc([N+](=O)[O-])cc1  1.968913
..                                               ...       ...
995                     COCOC(OC)C(c1ccccc1)c1ccccc1  2.081834
996            COCOC(OC)C(C#N)(c1ccccc1)c1ccc(OC)cc1  1.850059
997                       COCOCC(C)c1ccc(C(=O)OC)cc1  2.379702
998                   COCOC(Br)C(C)(C#N)c1ccc(OC)cc1  1.913486
999      COCOCC(C#N)(c1ccccc1)c1ccc([N+](=O)[O-])cc1  2.683121

[1000 rows x 2 columns]
     sssr    clogp        mr          mw   tpsa      chi0n     chi1n  \
0       2  4.09378   74.5810  301.010226  33.02   9.661630  5.613702   
1       3  5.52670  101.1664  397.031355  52.37  13.287527  7.800217   
2  

In [17]:
# Standardize data and perform principle component analysis
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

n_PC = 15
st = StandardScaler()
Xdata = st.fit_transform(X)

pca = PCA(n_components=n_PC)
Xdata = pca.fit_transform(Xdata)

In [18]:
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel
from sklearn.gaussian_process import GaussianProcessRegressor
from scipy.stats import norm
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C ,WhiteKernel as Wht,Matern as matk

def gpregression(Xtrain,Ytrain,Nfeature):    
    cmean=[1.0]*Nfeature
    cbound=[[1e-3, 1e3]]*Nfeature
    kernel = C(1.0, (1e-3,1e3)) * matk(cmean,cbound,1.5) + Wht(1.0, (1e-3, 1e3))
    gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=40, normalize_y=False)
    gp.fit(Xtrain, Ytrain)
    return gp

# Predict result using GP regression model
def gprediction(gpnetwork,xtest):
    y_pred, sigma = gpnetwork.predict(xtest, return_std=True)
    return y_pred, sigma

# Compute expected improvement
def expectedimprovement(xdata,gpnetwork,ybest,itag,epsilon):
    ye_pred, esigma = gprediction(gpnetwork, xdata)
    expI = np.empty(ye_pred.size, dtype=float)
    for ii in range(0,ye_pred.size):
        if esigma[ii] > 0:
            zzval=itag*(ye_pred[ii]-ybest)/float(esigma[ii])
            expI[ii]=itag*(ye_pred[ii]-ybest-epsilon)*norm.cdf(zzval)+esigma[ii]*norm.pdf(zzval)
        else:
            expI[ii]=0.0
    return expI

In [19]:
#Bayesian optimization 
def numberofopt(Xdata,Ydata,Xinfo,ndata,nPC):
    itag = 1
    epsilon = 0.01 # Control exploration/exploitation balance
    ntrain = 10 # Number of initial training data points
    nremain = ndata - ntrain
    dataset = np.random.permutation(ndata)
    a1data = np.empty(ntrain, dtype=int)
    a2data = np.empty(nremain, dtype=int)
    a1data[:] = dataset[0:ntrain]
    a2data[:] = dataset[ntrain:ndata]

    Xtrain = np.ndarray(shape=(ntrain, nPC), dtype=float)
    Xtraininfo = np.chararray(ntrain, itemsize=100)
    Ytrain = np.empty(ntrain, dtype=float)
    Xtrain[0:ntrain, :] = Xdata[a1data, :]
    Xtraininfo[0:ntrain] = Xinfo[a1data]
    Ytrain[0:ntrain] = Ydata[a1data]
    
    yoptLoc = np.argmax(Ytrain)
    yopttval = Ytrain[yoptLoc]
    xoptval = Xtraininfo[yoptLoc]
    yoptstep=0
    yopinit = yopttval
    xoptint = xoptval

    Xremain = np.ndarray(shape=(nremain, nPC), dtype=float)
    Xremaininfo = np.chararray(nremain, itemsize=100)
    Yremain = np.empty(nremain, dtype=float)
    Xremain[0:nremain, :] = Xdata[a2data, :]
    Xremaininfo[0:nremain] = Xinfo[a2data]
    Yremain[0:nremain] = Ydata[a2data]
    
    print('*** Initial training set ***')
    print(115*'-')
    print('{:<5s}{:<80s}{:<15s}'.format('Id','SMILES','Eox'))
    print(115*'-')
    for i in range(ntrain):
        if opt=='max':
            print('{:<5d}{:<80s}{:<15f}'.format(i,Xtraininfo[i].decode(),Ytrain[i]))
        elif opt=='min':
            print('{:<5d}{:<80s}{:<15f}'.format(i,Xtraininfo[i].decode(),-Ytrain[i]))
    print(115*'-')  

    print("Total number of inital training points: ", ntrain)
    if opt=='max':
        print("Initial best SMILES is "+xoptval.decode()+' with Eox = '+str(yopttval))
    elif opt=='min':
        print("Initial best SMILES is "+xoptval.decode()+' with Eox = '+str(-yopttval))

    for ii in range(0, Niteration):
        print('------------')
        print('Iteration '+str(ii+1))
        print('Training GPR model on '+str(ntrain+ii)+' data points')
        gpnetwork = gpregression(Xtrain, Ytrain, nPC)
        
        print('Making Eox prediction on the remaining '+str(ndata-ntrain-ii)+' data points')
        yt_pred, tsigma = gprediction(gpnetwork, Xtrain)
        
        ybestloc = np.argmax(Ytrain) # The current best y value
        ybest = yt_pred[ybestloc]
        ytrue = Ytrain[ybestloc]
        
        if yopttval < ytrue:
            yopttval = ytrue
            xoptval = Xtraininfo[ybestloc]
            
        print('Computing EI on the remaining '+str(ndata-ntrain-ii)+' data points')
        expI = expectedimprovement(Xremain, gpnetwork, ybest, itag, epsilon)
        expImax = np.max(expI)
        expimaxloc = np.argmax(expI)
        print('Based on EIs, the next SMILES to be evaluated and added to the training set is '+Xremaininfo[expimaxloc].decode())
    
        
        xnew = np.append(Xtrain, Xremain[expimaxloc]).reshape(-1, nPC)
        xnewinfo = np.append(Xtraininfo, Xremaininfo[expimaxloc])
        ynew = np.append(Ytrain, Yremain[expimaxloc])
        xrnew = np.delete(Xremain, expimaxloc, 0)
        xrnewinfo = np.delete(Xremaininfo, expimaxloc)
        yrnew = np.delete(Yremain, expimaxloc)
        if ii==0:
            Xexplored=Xremaininfo[expimaxloc]
            Yexplored=Yremain[expimaxloc]
        else:
            Xexploredtemp=np.append(Xexplored, Xremaininfo[expimaxloc])
            Yexploredtemp=np.append(Yexplored, Yremain[expimaxloc])
            del Xexplored,Yexplored
            Xexplored=Xexploredtemp
            Yexplored=Yexploredtemp
        del Xtrain, Ytrain, Xremaininfo, gpnetwork
        Xtrain = xnew
        Xtraininfo = xnewinfo
        Ytrain = ynew
        Xremain = xrnew
        Xremaininfo = xrnewinfo
        Yremain = yrnew
        del xnew, xnewinfo, ynew, xrnew, xrnewinfo, yrnew

    if not yopinit==yopttval:
        yoptstep = np.argmax(Yexplored) + 1       
    else:
        yoptstep=0
    dataorder = np.argsort(Yexplored)
    Yexploredtemp=Yexplored[dataorder]
    Xexploredtemp = Xexplored[dataorder]
    print('*** Summary ***')
    print(115*'-')
    print('{:<15s}{:<80s}{:<15s}'.format('Iteration','SMILES','Eox'))
    print(115*'-')
    for i,sml in enumerate(Xexplored):
        if opt=='max':
            print('{:<15d}{:<80s}{:<15f}'.format(i+1,sml.decode(),Yexplored[i]))
        elif opt=='min':
            print('{:<15d}{:<80s}{:<15f}'.format(i+1,sml.decode(),-Yexplored[i]))

    print(115*'-')  
    print("\n")
    if opt=='min':
        yopttval*=-1
    print("The best SMILES is "+xoptval.decode()+" with Eox = "+str(yopttval)+", which was found in iteration "+str(yoptstep))
    return xoptint,yopinit,xoptval,yopttval

In [20]:
#------- Program starts here -------------
opt = 'max' 
if opt=='max':
    Ydata = df.Eox.values 
elif opt=='min':
    Ydata = -1*df.Eox.values 
print('*** Finding SMILES with '+opt+'imum Eox ***')
ndata = len(Ydata)
print("Original shape of X and Y :",np.shape(Xdata),np.shape(Ydata))
Nruns = 1
Niteration = 30          # number of iteration in a given Bayesian  Optimization
Xinitguess = np.chararray(Nruns, itemsize=100)
Yinitguess = np.empty(Nruns, dtype=float)
Xoptimal = np.chararray(Nruns, itemsize=100)
Yoptimal = np.empty(Nruns, dtype=float)

for ii in range(0,Nruns):
    Xinitguess[ii], Yinitguess[ii], Xoptimal[ii], Yoptimal[ii] =numberofopt(Xdata, Ydata, Xinfo, ndata, n_PC)

*** Finding SMILES with maximum Eox ***
Original shape of X and Y : (1000, 15) (1000,)
*** Initial training set ***
-------------------------------------------------------------------------------------------------------------------
Id   SMILES                                                                          Eox            
-------------------------------------------------------------------------------------------------------------------
0    COCOC(Br)C(C#N)(c1ccccc1)c1ccc(C(=O)OC)cc1                                      2.503099       
1    CCc1ccc(C(C#N)(C#N)C(Oc2ccccc2)c2ccccc2)cc1                                     2.033319       
2    COC(Br)Cc1ccc([N+](=O)[O-])cc1                                                  3.050945       
3    COC(OC)C(C)(C#N)c1ccccc1                                                        2.487290       
4    COCOC(C#N)C(C#N)(c1ccccc1)c1ccc(C(=O)OC)cc1                                     2.650523       
5    COCOC(Br)Cc1ccc(C(=O)OC)cc1              